In [8]:
from qdrant_client import QdrantClient
import requests
import json
client = QdrantClient(host="10.10.10.5", grpc_port=6334, prefer_grpc=True)

In [9]:
client.list_full_snapshots()

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNAVAILABLE: ipv4:10.10.10.5:6334: Connection refused"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"failed to connect to all addresses; last error: UNAVAILABLE: ipv4:10.10.10.5:6334: Connection refused", grpc_status:14, created_time:"2024-07-04T07:19:54.5318287+00:00"}"
>

In [ ]:
# curl -X POST "http://localhost:6333/collections/your_collection_name/snapshots"
client.create_snapshot(collection_name=collection_name)

In [ ]:
# curl -X POST "http://target_host:6333/collections/your_collection_name/snapshots/recover" \
# -H "Content-Type: application/json" \
# -d '{"snapshot_path": "/path/to/target_backup_directory/snapshot_file"}'

In [ ]:
'http://10.10.10.5:6333/snapshots/full-snapshot-2024-06-18-07-55-26.snapshot'

In [44]:
client_1 = QdrantClient(host="47.110.156.41", grpc_port=6334, prefer_grpc=True)
client_1.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='行业公司名简介_25k'), CollectionDescription(name='places_bank'), CollectionDescription(name='专利_w2v'), CollectionDescription(name='my_collection'), CollectionDescription(name='企业_w2v_lda'), CollectionDescription(name='专利_bge_189')])

In [10]:
# collection_name='places_bank'
# client_1.recover_snapshot(
#     collection_name,
#     "E:/Downloads/places_bank-4931977184024862-2024-06-18-07-44-25.snapshot",#file:///
# )

In [ ]:
#curl -X POST "http://localhost:6333/snapshots/{snapshot_id}/restore"

In [22]:

# curl -X POST http://localhost:6333/collections/places_bank/snapshots/upload \
#      -H "api-key: your_api_key" \
#      -F snapshot=@E:\Downloads\places_bank-4931977184024862-2024-06-18-07-44-25.snapshot

In [29]:

file_path="E:\Downloads\places_bank-4931977184024862-2024-06-18-07-44-25.snapshot"
with open(file_path, 'rb') as f:
    files = {'snapshot': f}
    #headers={"Content-Type": "multipart/form-data"}
    res=requests.post("http://47.110.156.41:6333/collections/places_bank/snapshots/upload",
                files=files,
                #   headers= headers,
                #   data=json.dumps({"snapshot_path": 
                #                    "E:/Downloads/places_bank-4931977184024862-2024-06-18-07-44-25.snapshot"})
                )
res

<Response [200]>

In [33]:
res.text,res.content,res.json()

('{"result":true,"status":"ok","time":0.469759098}',
 b'{"result":true,"status":"ok","time":0.469759098}',
 {'result': True, 'status': 'ok', 'time': 0.469759098})

In [45]:
def recover_snapshots_upload(host,collection_name,file_path):
    url=f"http://{host}:6333/collections/{collection_name}/snapshots/upload"
    with open(file_path, 'rb') as f:
        files = {'snapshot': f}
        #headers={"Content-Type": "multipart/form-data"}
        response=requests.post(url,files=files)

    return response.json()

In [ ]:
# client.recover_snapshot(
#     "{collection_name}",
#     "http://example.com/path/to/snapshot.shapshot",
# )#file:///qdrant/snapshots/test_collection-2022-08-04-10-49-10.snapshot
r"E:\Downloads\places_bank-4931977184024862-2024-06-18-07-44-25.snapshot"

In [37]:
client.create_snapshot(collection_name='企业_w2v_lda')

SnapshotDescription(name='企业_w2v_lda-4931977184024862-2024-06-18-10-06-45.snapshot', creation_time='2024-06-18T10:06:46', size=79509504, checksum=None)

In [40]:
url='http://10.10.10.5:6333/collections/企业_w2v_lda/snapshots/企业_w2v_lda-4931977184024862-2024-06-18-10-06-45.snapshot'

In [44]:
recover_snapshots_upload('47.110.156.41','企业_w2v_lda',
                         "E:\Downloads\企业_w2v_lda-4931977184024862-2024-06-18-10-06-45.snapshot")

{'result': True, 'status': 'ok', 'time': 0.710806932}

In [45]:
client.create_snapshot(collection_name='专利_w2v')

SnapshotDescription(name='专利_w2v-4931977184024862-2024-06-19-01-49-08.snapshot', creation_time='2024-06-19T01:49:25', size=801428992, checksum=None)

In [57]:
collection_name='专利_w2v'
snapshots_name=client.list_snapshots(collection_name)[0].name
url=f'http://10.10.10.5:6333/collections/{collection_name}/snapshots/{snapshots_name}'
url

'http://10.10.10.5:6333/collections/专利_w2v/snapshots/专利_w2v-4931977184024862-2024-06-19-01-49-08.snapshot'

In [56]:
client_1.recover_snapshot(
    collection_name,
    location=url,
    wait=True
)

ResponseHandlingException: timed out

In [46]:
def download_file(url, local_filename):
    """
    Downloads a file from the given URL and saves it to the local system.
    """
    with requests.get(url, stream=True) as response:
        response.raise_for_status()
        with open(local_filename, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
    return local_filename


In [ ]:
local_filename=f'E:\Downloads\{snapshots_name}'
download_file(url, local_filename)

In [61]:
recover_snapshots_upload('47.110.156.41', collection_name, local_filename)

('47.110.156.41', '企业_w2v_lda')

In [84]:
client.embedding_model_name

'BAAI/bge-small-en'

In [51]:
#client.list_snapshots(collection_name)
client.delete_snapshot(collection_name,'专利_w2v_188_37-4931977184024862-2024-06-26-08-00-39.snapshot')

True

In [54]:
#client.list_full_snapshots()
client.delete_full_snapshot(snapshot_name='full-snapshot-2024-06-19-02-44-02.snapshot')

True

In [49]:
collection_name='专利_w2v_188_37'
client.create_snapshot(collection_name=collection_name)
snapshots_name=client.list_snapshots(collection_name)[0].name
url=f'http://10.10.10.5:6333/collections/{collection_name}/snapshots/{snapshots_name}'
local_filename=f'E:\Downloads\{snapshots_name}'
download_file(url, local_filename)
recover_snapshots_upload('47.110.156.41', collection_name, local_filename)
client.delete_snapshot(
collection_name=collection_name, snapshot_name=snapshots_name
)
#     to_client.recover_snapshot(
#     collection_name,
#     location=url,
#     wait=True
# )
  
client_1.count(collection_name)

KeyboardInterrupt: 

In [83]:
for collection  in client.get_collections().collections:
    snapshots_names=client.list_snapshots(collection.name)
    print(collection.name ,snapshots_names)
    if snapshots_names:
        snapshots_name=snapshots_names[0].name
        client.delete_snapshot(
        collection_name=collection.name, snapshot_name=snapshots_name)

企业_w2v_lda []
专利_w2v []
专利_w2v_188_37 []
places_bank []


In [86]:
#client.create_full_snapshot()
client.list_full_snapshots()

[]

In [5]:
import pickle
import numpy as np

def read_pickle_in_batches(file_path, batch_size):
    with open(file_path, 'rb') as f:
        while True:
            batch = []
            try:
                for _ in range(batch_size):
                    batch.append(pickle.load(f))
            except EOFError:
                break
            yield batch

def read_npy_in_batches(file_path, batch_size):
    data = np.load(file_path, mmap_mode='r')
    total_rows = data.shape[0]
    
    for i in range(0, total_rows, batch_size):
        yield data[i:i + batch_size]

In [41]:
pkl_path = 'Y:\Downloads\Patent\融资公司专利2024_payloads.pkl'
npy_path = 'Y:\Downloads\Patent\融资公司专利2024_bge_embeddings.npy'
batch_size = 1000
pkl_batches = read_pickle_in_batches(pkl_path, batch_size)
npy_batches = read_npy_in_batches(npy_path, batch_size)


# while True:
#     try:
#         pkl_batch = next(pkl_batches)
#         npy_batch = next(npy_batches)
#         print(f"Processing batch of size {len(pkl_batch)}")
#         for pkl_item, npy_item in zip(pkl_batch, npy_batch):
#             print(f"Pickle: {pkl_item}, Numpy: {npy_item}")

#     except StopIteration:
#         break


In [40]:
del pkl_batches
del npy_batches

NameError: name 'pkl_batches' is not defined

In [36]:
i=0
while True:
    try:
        pkl_batch = next(pkl_batches)
        npy_batch = next(npy_batches)
        size=len(pkl_batch)
        index = list(range(i,i + size))
        print(f"{i},Processing batch of size {size,len(npy_batch)}")
        points=Batch(ids= index,payloads=pkl_batch,vectors=npy_batch.tolist())
        client_1.upsert(collection_name=collection_name,points = points)     
        i+=size

    except StopIteration:
        break

In [7]:
with open(pkl_path, 'rb') as f:
    pkls=pickle.load(f)

In [9]:
pkls[0]

{'Co': '重庆钢铁股份有限公司',
 '序号': 123,
 '申请号': 'CN93117175.X',
 '公开（公告）号': 'CN1050661C',
 '标题 (中文)': '竖式烧结矿冷却机'}

In [8]:
len(pkls)

1897517

In [3]:
from qdrant_client.models import Batch,Distance,Batch,VectorParams,PointStruct

In [31]:
collection_name='专利_bge_189'
client_1.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE))

C:\Users\Administrator\AppData\Local\Temp\ipykernel_28544\1782672481.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client_1.recreate_collection(


True

In [15]:
del npy_batches

In [21]:
client_1=QdrantClient(host="47.110.156.41", grpc_port=6334, prefer_grpc=True)

In [32]:
client_1.count(collection_name=collection_name)

CountResult(count=0)

In [19]:
i,1896000

237000

In [27]:
len(pkls[1897000:1897000 + batch_size])

517

In [42]:
%%time
for i in range(0, len(pkls), batch_size):
    # if i<=237000:
    #     continue
    try:
        payload = pkls[i:i + batch_size]
        npy_batch = next(npy_batches)
        size=len(payload)
        index = list(range(i,i + size))
        print(f"{i},Processing batch of size {size,len(npy_batch)}")
        points=Batch(ids= index,payloads=payload,vectors=npy_batch.tolist())
        client_1.upsert(collection_name=collection_name,points = points)     
        #   points = [PointStruct(id=id_max+idx, vector=wo.wv[word].tolist(), 
        #                 payload={"word": word,"行业": col,'index':idx,
        #                          'tf':tf.get(word,1),
        #                          'df':df.get(word,1)})
        #         for word, idx in batch]
    except StopIteration:
        break

0,Processing batch of size (1000, 1000)
1000,Processing batch of size (1000, 1000)
2000,Processing batch of size (1000, 1000)
3000,Processing batch of size (1000, 1000)
4000,Processing batch of size (1000, 1000)
5000,Processing batch of size (1000, 1000)
6000,Processing batch of size (1000, 1000)
7000,Processing batch of size (1000, 1000)
8000,Processing batch of size (1000, 1000)
9000,Processing batch of size (1000, 1000)
10000,Processing batch of size (1000, 1000)
11000,Processing batch of size (1000, 1000)
12000,Processing batch of size (1000, 1000)
13000,Processing batch of size (1000, 1000)
14000,Processing batch of size (1000, 1000)
15000,Processing batch of size (1000, 1000)
16000,Processing batch of size (1000, 1000)
17000,Processing batch of size (1000, 1000)
18000,Processing batch of size (1000, 1000)
19000,Processing batch of size (1000, 1000)
20000,Processing batch of size (1000, 1000)
21000,Processing batch of size (1000, 1000)
22000,Processing batch of size (1000, 1000)
2

In [11]:
220713221//1024//8

26942

In [12]:
220713221/33554432

6.577766567468643

In [2]:
from gensim import corpora
from gensim.models import Word2Vec

In [3]:
from qdrant_client.models import Batch,Distance,VectorParams

In [4]:
import pandas as pd
word_tf_df=pd.read_csv('Y:\Downloads\Patent\patent_word_tf_df_188.csv',index_col=0)
tf=word_tf_df.tf.to_dict()
df=word_tf_df.df.to_dict()

In [5]:
suffix='188_37'
wo = Word2Vec.load(f'Y:\Downloads\Patent\patent_w2v_{suffix}.model') 

In [6]:
index=list(range(len(wo.wv)))
payload=[{"word": word,'index':idx,'tf':tf.get(word,1),'df':df.get(word,1)} for word,idx in wo.wv.key_to_index.items()]
points=Batch(
    ids=index,
    vectors=wo.wv.vectors.tolist(),
    payloads=payload
)

In [11]:
client = QdrantClient(host="47.110.156.41", grpc_port=6334, prefer_grpc=True)

In [9]:
collection_name=f'专利_w2v_{suffix}'
client.create_collection(collection_name,vectors_config=VectorParams(size=wo.wv.vectors.shape[1],distance=Distance.COSINE))
client.upsert(collection_name=collection_name,points = points) 

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [10]:
client.count(collection_name)

CountResult(count=369401)

In [11]:
client.get_collection(collection_name)

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=369401, points_count=369401, segments_count=4, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=250, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=

In [12]:
del payload
del points
del wo

In [13]:
collection_name='行业公司名简介_25k'
client.count('行业公司名简介_25k')

CountResult(count=50826)

In [14]:
50826/2

25413.0

In [16]:
client.delete_payload(
    collection_name=collection_name,
    keys=["profile"],
    points=list(range(25413,50826)),
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [19]:
client.retrieve(
    collection_name=collection_name,
    ids=[50825,25413,25412],
)

[Record(id=50825, payload={'行业': '金融科技', '工商全称': '爱因互动科技发展（北京）有限公司', '公司序号': 25511, '公司简称': '爱因互动'}, vector=None, shard_key=None),
 Record(id=25413, payload={'行业': '传统制造', '工商全称': '天津蔚蓝科技有限公司', '公司序号': 0, '公司简称': 'SciXair空气净化器'}, vector=None, shard_key=None),
 Record(id=25412, payload={'行业': '金融科技', '工商全称': '爱因互动科技发展（北京）有限公司', '公司序号': 25511, '公司简称': '爱因互动'}, vector=None, shard_key=None)]

In [3]:
import pickle

In [22]:
with open('Y:\Downloads\Patent\行业公司名简介2024_payloads.pkl', 'rb') as file:
    playload = pickle.load(file)

In [24]:
len(playload)

50826

In [25]:
playload[0]

{'公司序号': 0,
 '公司简称': 'SciXair空气净化器',
 '工商全称': '天津蔚蓝科技有限公司',
 '行业': '传统制造',
 '简介长度': 64.0}

In [26]:
playload[0]['简介长度']

64.0

In [29]:
client.count('行业公司名简介_25k')

CountResult(count=50826)

In [33]:
playload[5951]

{'公司序号': 5951,
 '公司简称': '连山融资项目',
 '工商全称': '上海富捷信息技术有限公司',
 '行业': '金融',
 '简介长度': nan}

In [36]:
playload[5951]['简介长度']>5

False

In [37]:
for i,p in enumerate(playload):
    if i<5951:
        continue
    if '简介长度' in p and p['简介长度']>5:
        client.set_payload(
            collection_name=collection_name,
            payload={
                '简介长度': p['简介长度'],
            },
            points=[i],
        )

In [28]:
i

5951

In [2]:
from qdrant_client.models import  SetPayloadOperation,SetPayload

operation=SetPayloadOperation(
            set_payload=SetPayload(
                payload={
                    "test_payload_2": 2,
                    "test_payload_3": 3,
                },
                points=[1],
            )
)
operation

SetPayloadOperation(set_payload=SetPayload(payload={'test_payload_2': 2, 'test_payload_3': 3}, points=[1], filter=None, shard_key=None, key=None))

In [5]:
with open('Y:\Downloads\Patent\融资公司专利2024_payloads_1.pkl', 'rb') as file:
    playload = pickle.load(file)

In [6]:
len(playload),playload[0]

(1897517, {'摘要长度': 160, 'table': 'patent202210'})

In [49]:
for i,p in enumerate(playload[:10]):
    operation=SetPayloadOperation(
        set_payload=SetPayload(
            payload=p if p.get('摘要长度',0)>0 else {'table':p.get('table')} ,
            points=[i],
        )
    )
operation

SetPayloadOperation(set_payload=SetPayload(payload={'摘要长度': 172, 'table': 'patent202210'}, points=[9], filter=None, shard_key=None, key=None))

In [8]:
update_operations=[SetPayloadOperation(
                        set_payload=SetPayload(
                            payload=p if p.get('摘要长度',0)>0 else {'table':p.get('table')} ,
                            points=[i],
                        )
                    ) for  i,p in enumerate(playload)]
len(update_operations),update_operations[0]

(1897517,
 SetPayloadOperation(set_payload=SetPayload(payload={'摘要长度': 160, 'table': 'patent202210'}, points=[0], filter=None, shard_key=None, key=None)))

In [19]:
collection_name='专利_w2v'
client.count(collection_name)

CountResult(count=594225)

In [12]:
# %%time
# client.batch_update_points(collection_name=collection_name,update_operations=update_operations,)

In [104]:
from qdrant_client.models import PayloadFieldSchema,PayloadSchemaType,PayloadSchemaParams,VectorParams,IntegerIndexParams,TextIndexParams,VectorsConfigDiff,VectorsConfig,SparseVectorsConfig,VectorParamsDiff,ScalarQuantization,ScalarQuantizationConfig,CollectionParamsDiff,OptimizersConfigDiff,VectorsConfigDiff,HnswConfigDiff,HnswConfig,CollectionParams,CollectionConfig
from qdrant_client.models import Filter, PayloadField,FieldCondition, HasIdCondition, IsEmptyCondition, MatchValue,SparseVectorParams,Distance,ScalarType,UpdateCollection#稀疏向量索引

In [ ]:
hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None)
#vectors=VectorParams(size=1024, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None), 
#config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1024, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None), 

In [ ]:
#search_params=models.SearchParams(hnsw_ef=128, exact=False),

In [21]:
client.get_collection('专利_w2v')

CollectionInfo(status=<CollectionStatus.YELLOW: 'yellow'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=594225, points_count=594225, segments_count=4, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=250, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=True, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config

In [38]:
#If true, vectors are served from disk, improving RAM usage at the cost of latency
#启用磁盘向量和 HNSW 索引
vectors_config=VectorParams(
        size=768, distance=Distance.COSINE, on_disk=True
    )

In [54]:
# collection_name: str, 
# optimizers_config:qdrant_client.conversions.common_types.OptimizersConfigDiff | None = None, 
# collection_params: qdrant_client.conversions.common_types.CollectionParamsDiff | None = None, 
# vectors_config: qdrant_client.conversions.common_types.VectorsConfigDiff | None = None,
# hnsw_config: qdrant_client.conversions.common_types.HnswConfigDiff | None = None, 
# quantization_config: qdrant_client.conversions.common_types.QuantizationConfigDiff | None = None, 
# timeout: int | None = None,

In [20]:
client.update_collection(
    collection_name="专利_w2v",hnsw_config=HnswConfigDiff(on_disk=True))#向量索引
#vectors_config=

True

In [50]:
client.update_collection(
collection_name="专利_bge_189",hnsw_config=HnswConfigDiff(on_disk=True) ,)

True

In [57]:
VectorsConfigDiff.__dict__

{'_inst': False,
 '_name': 'Dict',
 '__origin__': dict,
 '__slots__': None,
 '__args__': (str, ForwardRef('VectorParamsDiff')),
 '__parameters__': ()}

In [69]:
VectorParamsDiff(on_disk=True)#vectors_config=

VectorParamsDiff(hnsw_config=None, quantization_config=None, on_disk=True)

In [77]:
VectorParamsDiff(on_disk=True).dict()

{'hnsw_config': None, 'quantization_config': None, 'on_disk': True}

In [125]:
from typing import Dict
vectors_config_diff = VectorsConfigDiff({"on_disk": VectorParamsDiff(on_disk=True)})

TypeError: Type Dict cannot be instantiated; use dict() instead

In [39]:
#Memmap 存储 - 创建与磁盘上的文件关联的虚拟地址空间，映射文件不会直接加载到 RAM 中。相反，它们使用页面缓存来访问文件的内容。 此方案允许灵活使用可用内存。有了足够的RAM，它几乎和内存存储一样快。
#每个段要存储内存中的矢量的最大大小（以千字节为单位）。大于此阈值的段将存储为只读 memmaped 文件。
#默认情况下，Memmap 存储处于禁用状态，要启用它，请将此阈值设置为合理的值。
OptimizersConfigDiff(memmap_threshold=20000)#全局设置阈值

OptimizersConfigDiff(deleted_threshold=None, vacuum_min_vector_number=None, default_segment_number=None, max_segment_size=None, memmap_threshold=20000, indexing_threshold=None, flush_interval_sec=None, max_optimization_threads=None)

In [43]:
scalar=ScalarQuantizationConfig(
            type=ScalarType.INT8,
            always_ram=True,
        )
scalar#量化将向量转换为更紧凑的表示形式，可以存储在内存中并用于搜索。使用较小的向量

ScalarQuantizationConfig(type=<ScalarType.INT8: 'int8'>, quantile=None, always_ram=True)

In [73]:
ScalarQuantization(
        scalar=scalar,
        )

ScalarQuantization(scalar=ScalarQuantizationConfig(type=<ScalarType.INT8: 'int8'>, quantile=None, always_ram=True))

In [96]:
client.update_collection(
collection_name="places_bank",vectors_config=VectorParamsDiff(on_disk=True),
quantization_config=ScalarQuantization(scalar=ScalarQuantizationConfig(type=ScalarType.INT8,always_ram=True,)),)
                                       

TypeError: Message must be initialized with a dict: qdrant.UpdateCollection

In [127]:
payload = json.dumps({
    "vectors": {
        "": {
            "on_disk": True
        }
    },
    "quantization_config": {
        "scalar": {
            "type": "int8",
            "always_ram": True #如果为 true - 量化的向量将始终存储在 RAM 中，忽略主存储的配置
        }
    }
})
headers = {"Content-Type": "application/json"}

In [124]:
host='47.110.156.41'
collection_name='places_bank'
response = requests.patch(f'http://{host}:6333/collections/{collection_name}',headers =headers, data=payload)
response.json()

{'result': True, 'status': 'ok', 'time': 0.015046173}

In [129]:
client.search('places_bank',query_vector=[30,120.8])

[ScoredPoint(id=20, version=0, score=0.3438778221607208, payload={'bank_name': '中国光大银行股份有限公司宁波余姚支行', 'bank': '光大银行', 'area': 'ningbo'}, vector=None, shard_key=None),
 ScoredPoint(id=3, version=0, score=0.3461451232433319, payload={'area': 'ningbo', 'bank': '平安银行', 'bank_name': '平安银行宁波余姚支行'}, vector=None, shard_key=None),
 ScoredPoint(id=18, version=0, score=0.37911152839660645, payload={'bank_name': '中国光大银行股份有限公司宁波余慈支行', 'bank': '光大银行', 'area': 'ningbo'}, vector=None, shard_key=None),
 ScoredPoint(id=5, version=0, score=0.491935670375824, payload={'bank_name': '平安银行宁波慈溪支行', 'bank': '平安银行', 'area': 'ningbo'}, vector=None, shard_key=None),
 ScoredPoint(id=26, version=0, score=0.5124711394309998, payload={'bank_name': '中国光大银行股份有限公司宁波慈溪支行', 'bank': '光大银行', 'area': 'ningbo'}, vector=None, shard_key=None),
 ScoredPoint(id=11, version=0, score=0.6754504442214966, payload={'area': 'ningbo', 'bank': '平安银行', 'bank_name': '平安银行宁波奉化支行'}, vector=None, shard_key=None),
 ScoredPoint(id=19, version=0,

In [130]:
client.get_collection('places_bank')

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=0, points_count=54, segments_count=3, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=2, distance=<Distance.EUCLID: 'Euclid'>, hnsw_config=None, quantization_config=None, on_disk=True, datatype=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=32, ef_construct=200, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=ScalarQuant

In [126]:
collection_name='专利_bge_189'
response = requests.patch(f'http://{host}:6333/collections/{collection_name}',headers =headers, data=payload)
response.json()

{'result': True, 'status': 'ok', 'time': 0.008995901}

In [132]:
client.update_collection(
collection_name="专利_bge_189",hnsw_config=HnswConfigDiff(on_disk=False) ,)

True

In [136]:
client.get_collection('专利_bge_189')

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=1897517, points_count=1897517, segments_count=12, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1024, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=True, datatype=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_con

In [137]:
payload = json.dumps({
    "vectors": {
        "": {
            "on_disk": True
        }
    }
})
collection_name='专利_w2v'
response = requests.patch(f'http://{host}:6333/collections/{collection_name}',headers =headers, data=payload)
response.json()

{'result': True, 'status': 'ok', 'time': 0.006941608}

In [ ]:
client.create_payload_index(
    collection_name="{collection_name}",
    field_name="name_of_the_field_to_index",
    field_schema="keyword",
)

参数化索引  
IntegerIndexParams(type=models.IntegerIndexType.INTEGER,lookup=False,range=True,  
全文索引  
TextIndexParams(
        type="text",
        tokenizer=models.TokenizerType.WORD,
        min_token_len=2,
        max_token_len=15,
        lowercase=True,
    ),

In [19]:
client.create_payload_index(
    collection_name="行业公司名简介_25k",
    field_name="简介长度",
    field_schema=PayloadSchemaType("integer"),
)

UpdateResult(operation_id=25398, status=<UpdateStatus.COMPLETED: 'completed'>)

In [20]:
client.create_payload_index(
    collection_name="专利_bge_189",
    field_name="摘要长度",
    field_schema=PayloadSchemaType("integer"),
)

UpdateResult(operation_id=1899416, status=<UpdateStatus.COMPLETED: 'completed'>)

In [145]:
i2=client.retrieve(collection_name="专利_bge_189", ids=[2], with_vectors=True)
i2[0].vector

[0.03180668130517006,
 -0.020956439897418022,
 -0.00612410344183445,
 0.010814242996275425,
 0.01716265268623829,
 -0.001597241498529911,
 -0.02715976908802986,
 -0.01438081543892622,
 0.017202043905854225,
 0.020065171644091606,
 0.0020439752843230963,
 0.04171101748943329,
 0.010596370324492455,
 -0.03856956213712692,
 -0.030774783343076706,
 -0.00620531290769577,
 0.013026579283177853,
 0.011924387887120247,
 0.015517737716436386,
 -0.005444876383990049,
 0.020678961649537086,
 0.0033271973952651024,
 -0.008741476573050022,
 0.0113316485658288,
 0.03135519474744797,
 0.0014711013063788414,
 0.013645828701555729,
 0.002847616095095873,
 -0.002997361123561859,
 -0.0339239202439785,
 -0.005655971355736256,
 0.003792912932112813,
 -0.008928515017032623,
 0.0031156574841588736,
 -0.006073251832276583,
 -0.014159825630486012,
 0.01316780038177967,
 -0.007948610000312328,
 0.011621534824371338,
 -0.004912414588034153,
 -0.0009058211580850184,
 -0.002862044842913747,
 -0.0037436001002788544

In [146]:
client.search(collection_name="专利_bge_189",query_vector=i2[0].vector)

[ScoredPoint(id=2, version=1900, score=1.0298233032226562, payload={'申请号': 'CN94110662.4', '序号': 154, '摘要长度': 202, '标题 (中文)': '麦白霉素胃溶片的生产工艺技术', 'Co': '山东鲁抗医药股份有限公司', 'table': 'patent202210', '公开（公告）号': 'CN1104887A'}, vector=None, shard_key=None),
 ScoredPoint(id=189584, version=191482, score=0.9342460632324219, payload={'申请号': 'CN201310097794.3', '序号': 366016, '摘要长度': 177, '标题 (中文)': '一种艾普拉唑肠溶口崩片及其制备方法', 'Co': '丽珠医药集团股份有限公司', 'table': 'patent202210', '公开（公告）号': 'CN103169684B'}, vector=None, shard_key=None),
 ScoredPoint(id=189583, version=191481, score=0.9298553466796875, payload={'申请号': 'CN201310097715.9', '序号': 366015, '摘要长度': 181, '标题 (中文)': '一种艾普拉唑钠肠溶口崩片及其制备方法', 'Co': '丽珠医药集团股份有限公司', 'table': 'patent202210', '公开（公告）号': 'CN103169683B'}, vector=None, shard_key=None),
 ScoredPoint(id=844810, version=846708, score=0.9223175048828125, payload={'申请号': 'CN201910060247.5', '序号': 1357801, '摘要长度': 251, '标题 (中文)': '一种阿奇霉素干混悬剂', 'Co': '江西赣南海欣药业股份有限公司', 'table': 'patent202404', '公开（公告）号': 'CN10

In [141]:
client.create_payload_index(
    collection_name="专利_bge_189",
    field_name='序号',
    field_schema=PayloadSchemaType("integer"),
)

UpdateResult(operation_id=1899418, status=<UpdateStatus.COMPLETED: 'completed'>)

负载索引

In [ ]:
client.create_payload_index(
    collection_name="企业_w2v_lda",
    field_name="行业",
    field_schema=PayloadSchemaType("keyword"),
)

In [24]:
client.create_payload_index(
    collection_name="企业_w2v_lda",
    field_name="co",
    field_schema=PayloadSchemaType("keyword"),
)

UpdateResult(operation_id=6, status=<UpdateStatus.COMPLETED: 'completed'>)

In [18]:
client.create_payload_index(
    collection_name="专利_w2v_188_37",
    field_name="word",
    field_schema=PayloadSchemaType("keyword"),
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [16]:
client.create_payload_index(
    collection_name="专利_w2v",
    field_name="word",
    field_schema=PayloadSchemaType("keyword"),
)

UpdateResult(operation_id=117, status=<UpdateStatus.COMPLETED: 'completed'>)

In [17]:
client.create_payload_index(
    collection_name="专利_w2v",
    field_name="行业",
    field_schema=PayloadSchemaType("keyword"),
)

UpdateResult(operation_id=119, status=<UpdateStatus.COMPLETED: 'completed'>)

In [23]:
client.create_payload_index(
    collection_name="places_bank",
    field_name="bank",
    field_schema=PayloadSchemaType("keyword"),
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [25]:
client.update_collection(
    collection_name="places_bank",collection_params=CollectionParamsDiff(on_disk_payload=True)
)

True

In [26]:
client.update_collection(
    collection_name="企业_w2v_lda",collection_params=CollectionParamsDiff(on_disk_payload=True)
)

True

In [27]:
client.update_collection(
    collection_name="专利_w2v",collection_params=CollectionParamsDiff(on_disk_payload=True)
)

True

In [28]:
match=[IsEmptyCondition(is_empty=PayloadField(key='摘要长度'), )]
query_filter = Filter(must=[], must_not=match) 
client.count('专利_bge_189',count_filter=query_filter)

CountResult(count=1891374)

In [29]:
match=[IsEmptyCondition(is_empty=PayloadField(key='摘要长度'), )]
query_filter = Filter(must=match) 
client.count('专利_bge_189',count_filter=query_filter)

CountResult(count=6143)

In [31]:
match=[IsEmptyCondition(is_empty=PayloadField(key='简介长度'), )]
query_filter = Filter(must=match, must_not=[]) 
client.count('行业公司名简介_25k',count_filter=query_filter)

CountResult(count=25433)

In [13]:
client.get_collection('专利_bge_189')

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=1897517, points_count=1897517, segments_count=12, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1024, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_con

In [7]:
import requests
def qdrant_livez(host, https=False,api_key=''):
    url = f"https://{host}:6333/livez" if https else f"http://{host}:6333/livez" 
    headers = {"api-key": api_key} if api_key else {}
    try:
        response = requests.get(url, headers=headers, timeout=3)
        return response.text, response.status_code
    except requests.exceptions.RequestException as e:
        return e, 400
    except Exception as e:
        return e, 500

In [6]:
qdrant_livez('47.110.156.41', api_key='')

('healthz check passed', 200)